In [ ]:
!pip install transformers sentencepiece

In [ ]:
!pip install torchtext==0.6.0

In [ ]:
!pip install evaluate sacrebleu

In [ ]:
# Specify the filename of loss file txt
output_filename = "averageLosses.txt"

# Open the file in write mode
with open(output_filename, "w") as f:
    # Write a space character to the file
    f.write(" ")

print(f"A space character has been written to {output_filename}.")

In [ ]:
from transformers import AutoTokenizer, PegasusForConditionalGeneration, AdamW
import torch
import csv
import numpy as np
from tqdm import tqdm

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Read data from train.tsv and create training examples
training_examples = []

#TODO: REPLACE FOLLOWING WITH TRAINING DATA (TSV FILE) PATH
with open('/PATH/TO/TRAINING.TSV', 'r') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')
    next(reader)  # Skip header row
    for row in reader:
        fixed_sentence = row[0]
        buggy_sentence = row[1]
        training_examples.append((fixed_sentence, buggy_sentence))

# Load Pegasus tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")

model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

# Move models to GPU
model.to(device)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

#TODO: REPLACE THE FOLLOWING WITH LOSS FILE TXT PATH
loss_file = "/PATH/TO/averageLosses.txt"

load = False
if load:
    #TODO: REPLACE THE FOLLOWING WITH MODEL TRAINING CHECKPOINT .PT FILE
    model.load_state_dict(torch.load("/PATH/TO/pegasusModel.pt"))

# Training loop
batch_size = 16
average_losses = []
for epoch in range(30):  # Number of epochs
    total_loss = 0.0

    tokenized_example_subset = training_examples

    # Shuffle the tokenized examples
    np.random.shuffle(tokenized_example_subset)

    num_batches = len(tokenized_example_subset) // batch_size

    for batch_idx in tqdm(range(num_batches), desc=f"Epoch {epoch+1}"):
        start_idx = batch_idx * batch_size
        end_idx = start_idx + batch_size

        batch_examples = tokenized_example_subset[start_idx:end_idx]

        input_code_tensors = []
        target_code_tensors = []

        for example in batch_examples:
            tokenized_incorrect, tokenized_correct = example

            # Tokenize using the Pegasus tokenizer
            encoded_inputs = tokenizer(tokenized_incorrect, padding='max_length', max_length=150, return_tensors='pt', truncation=True)
            input_code_tensor = encoded_inputs.input_ids.to(device)

            encoded_targets = tokenizer(tokenized_correct, padding='max_length', max_length=150, return_tensors='pt', truncation=True)
            target_code_tensor = encoded_targets.input_ids.to(device)

            input_code_tensors.append(input_code_tensor)
            target_code_tensors.append(target_code_tensor)

        # Create batch tensors
        input_code_tensors = torch.cat(input_code_tensors, dim=0)
        target_code_tensors = torch.cat(target_code_tensors, dim=0)

        attention_masks = torch.where(input_code_tensors != 0, torch.tensor(1), input_code_tensors)

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=input_code_tensors, attention_mask=attention_masks, labels=target_code_tensors)
        loss = outputs.loss

        # Backward pass
        loss.backward()

        # Update parameters
        optimizer.step()

        total_loss += loss.item()

    # Save model checkpoint at every epoch
    #TODO REPLACE WITH TARGET SAVE LOCATION FOR THE MODEL .PT FILE
    torch.save(model.state_dict(), f"/PATH/TO/pegasusModel.pt")

    average_loss = total_loss / num_batches
    print(f"Saved: Epoch {epoch+1}: Average Loss = {average_loss}")
    average_losses.append(average_loss)

    #write epoch training loss to file
    with open(loss_file, "a") as f:
        f.write(str(average_loss) + ", \n")

print("Average training loss:")
print(average_losses)


In [ ]:
#Inference

from transformers import AutoTokenizer, PegasusForConditionalGeneration
import torch
import pandas as pd
import evaluate

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Pegasus tokenizer and model
pegasus_tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")
pegasus_model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

# Load the saved model checkpoint
#TODO: REPLACE THE FOLLOWING WITH MODEL PATH
model_checkpoint_path = "/PATH/TO/pegasusModel.pt"
pegasus_model.load_state_dict(torch.load(model_checkpoint_path, map_location=device))  # Use map_location to ensure it's loaded to the correct device
pegasus_model.to(device)  # Move the model to the correct device (GPU or CPU)
pegasus_model.eval()

# Load  test.tsv file
#TODO: REPLACE WITH TEST DATASET PATH
test_file_path = "/PATH/TO/Test.tsv"  # Replace with the actual file path
df = pd.read_csv(test_file_path, sep='\t')

total_score = 0
num_iterations = 0

# Create an output file for writing
# TODO: Specify the path where you want to save the file
output_file_path = "output_predictions.txt"  
with open(output_file_path, "w", encoding="utf-8") as output_file:
    for index, row in df.iterrows():
        # Sample input text
        sample_input = row["fix"]

        # Tokenize the input text
        input_ids = pegasus_tokenizer.encode(sample_input, return_tensors="pt")

        # Move inputs to the correct device (GPU or CPU)
        input_ids = input_ids.to(device)

        # Perform inference
        with torch.no_grad():
            output = pegasus_model.generate(input_ids, max_length=150, num_beams=5, early_stopping=True)

        # Convert the output token IDs back to text
        output_text = pegasus_tokenizer.decode(output[0], skip_special_tokens=True)
        target_text = row["bug"]

        # Calculate CHRF score
        prediction = [output_text]
        reference = [[target_text]]
        chrf = evaluate.load("chrf")
        results = chrf.compute(predictions=prediction, references=reference)
        score = results["score"]
        # Print the output
        print("Round:", index)
        print("Score:", score)
        print()

        # Write the predictions to the output file
        output_file.write("Round: {}\n".format(index))
        output_file.write("Input: {}\n".format(sample_input))
        output_file.write("Output: {}\n".format(output_text))
        output_file.write("Target: {}\n".format(target_text))
        output_file.write("CHRF Score: {}\n".format(score))
        output_file.write("\n")  # Add a blank line between entries
        
        total_score = total_score + score
        num_iterations = num_iterations + 1

print("Predictions saved to:", output_file_path)
print("Average CHRF score:", total_score/num_iterations)
